## We need to find set of REs that:
1. one of the REs must cut human mtDNA only one time
2. other REs must not cut mtDNA but must cut nuclear DNA to pieces less than 3-5kb

In [86]:
import re
import os
import random
import glob
from collections import Counter
from multiprocessing import Pool
from typing import Dict, List, Any

import pandas as pd
import matplotlib.pyplot as plt
from Bio.Restriction import Analysis, AllEnzymes, RestrictionBatch
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import tqdm

In [2]:
PIECES_SIZE = 5000
PATH_TO_HUMAN_GENOME = "../data/external/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/*.fna"
# PATH_TO_RE = "../data/processed/cuted_seqs_num.csv"
PATH_TO_REF_MT = "../data/external/NC_012920.1.fasta"
PATH_TO_SEQS_MT = "../data/raw/sequence.fasta"

In [3]:
len(AllEnzymes)

978

In [4]:
# human_genome = SeqIO.parse(PATH_TO_HUMAN_GENOME, "fasta")
mt_seqs = SeqIO.parse(PATH_TO_SEQS_MT, "fasta")
ref_mt = next(SeqIO.parse(PATH_TO_REF_MT, "fasta"))

## Search the enzymes that don't cut mtDNA

In [5]:
def extract_RE_without_site_on_mt(rec: SeqRecord) -> List[Dict]:
    ana = Analysis(AllEnzymes, rec.seq, linear=False)
    data = []
    for restr_enz in ana.without_site():
        re_name = repr(restr_enz)
        one_data = {"RE": re_name, "SeqName": rec.description}
        data.append(one_data)
    return data

In [7]:
# 27 min and 10GB of RAM
threads = 24
with Pool(threads) as p:
    collection_of_pot_rs = p.map(extract_RE_without_site_on_mt, mt_seqs)

In [11]:
pot_rs = []
for xx in collection_of_pot_rs:
    for x in xx:
        pot_rs.append(x)

re_without_site = pd.DataFrame(pot_rs)
re_without_site.to_csv("../data/interim/re_without_site.csv", index=None)
re_without_site.head()

,RE,SeqName
0,ArsI,MK968879.1 Homo sapiens isolate YHL_TK036_F4b1...
1,BglII,MK968879.1 Homo sapiens isolate YHL_TK036_F4b1...
2,FspI,MK968879.1 Homo sapiens isolate YHL_TK036_F4b1...
3,PspOMII,MK968879.1 Homo sapiens isolate YHL_TK036_F4b1...
4,MteI,MK968879.1 Homo sapiens isolate YHL_TK036_F4b1...


In [16]:
re_maximal_subset = re_without_site.RE.value_counts().reset_index()
re_maximal_subset.columns = ["RE", "NotCuttedSeqsNum"]
re_maximal_subset["Percentage"] = re_maximal_subset.NotCuttedSeqsNum / len(collection_of_pot_rs) * 100
re_maximal_subset.to_csv("../data/processed/NotCuttedSeqsNum.csv", index=None)
re_maximal_subset.head()

,RE,NotCuttedSeqsNum,Percentage
0,Sse232I,56445,100.0
1,SwaI,56445,100.0
2,Sse8387I,56445,100.0
3,MreI,56445,100.0
4,MauBI,56445,100.0


## Search of minimal subset of REs that destroy muclear DNA

1. initially need to add to the set ClaI, as RE that cut mtDNA only one time
2. drop RE dublicates
3. apply approach from GO hw2

In [5]:
re_maximal_subset = pd.read_csv("../data/processed/NotCuttedSeqsNum.csv")
re_maximal_subset

,RE,NotCuttedSeqsNum,Percentage
0,Sse232I,56445,100.000000
1,SwaI,56445,100.000000
2,Sse8387I,56445,100.000000
3,MreI,56445,100.000000
4,MauBI,56445,100.000000
...,...,...,...
255,Cma23826I,1,0.001772
256,Acc65V,1,0.001772
257,PpiP13II,1,0.001772
258,Sse8647I,1,0.001772


In [36]:
def collect_isoschizomers(enzymes: RestrictionBatch) -> List[List[str]]:
    """Search and collect all isoschizomers in passed enzymes"""
    data = []
    visited = set()
    for RE in enzymes:
        if repr(RE) in visited:
            continue

        visited.add(repr(RE))
        isosh_names = []
        for isosh in RE.isoschizomers():
            visited.add(repr(isosh))
            isosh_names.append(repr(isosh))
        data.append([repr(RE)] + isosh_names)
    return data


# # collect full collection of isoschizomers
# full_isosh = collect_isoschizomers(AllEnzymes)
# with open("../data/processed/full_isoschizomers.txt", "w") as fout:
#     for batch in full_isosh:
#         fout.write(",".join(batch) + "\n")

In [55]:
# sample RE that don't cut most of mtDNA
cutoff = 99.5 # %
REs_without_site_on_mt = re_maximal_subset[re_maximal_subset.Percentage > cutoff].RE.values
print(REs_without_site_on_mt.shape)

# create sample batch (custom data structure)
ClaI = AllEnzymes.get("ClaI")
excess_maximal_subset = RestrictionBatch(list(REs_without_site_on_mt) + ["ClaI"])
excess_maximal_subset

(78,)


RestrictionBatch(['AbsI', 'AcvI', 'AdeI', 'AgeI', 'ArsI', 'AscI', 'AsiGI', 'AsiSI', 'AspJHL3II', 'BbrPI', 'BoxI', 'BsePI', 'BshTI', 'Bsp460III', 'Bsp68I', 'BspGI', 'BssHII', 'BstPAI', 'BtuMI', 'CciNI', 'Cfr9I', 'ClaI', 'CpoI', 'CspAI', 'CspI', 'DraIII', 'Ecl35734I', 'Eco72I', 'FseI', 'FspAI', 'GauT27I', 'Lmo370I', 'MauBI', 'McaTI', 'MluI', 'MreI', 'MspSC27II', 'MteI', 'NotI', 'NpeUS61II', 'NruI', 'PalAI', 'PauI', 'PinAI', 'Ple19I', 'PmaCI', 'PmlI', 'PshAI', 'PspCI', 'PspXI', 'Pst273I', 'PteI', 'PvuI', 'RgaI', 'RigI', 'RpaB5I', 'RruI', 'Rsr2I', 'RsrII', 'SbfI', 'SdaI', 'SfaAI', 'SfiI', 'SgfI', 'SgrDI', 'SgsI', 'SmaI', 'SmiI', 'SrfI', 'Sse232I', 'Sse8387I', 'Ssp714II', 'SstE37I', 'Sth20745III', 'SwaI', 'TspARh3I', 'TspMI', 'UbaF13I', 'XmaI'])

In [56]:
# collect collection of isoschizomers for used sample REs
sufficient_sample = []
full_isosh = collect_isoschizomers(excess_maximal_subset)
with open(f"../data/processed/used_re_isoschizomers_{cutoff}%.txt", "w") as fout:
    for batch in full_isosh:
        sufficient_sample.append(batch[0])
        fout.write(",".join(batch) + "\n")

print(len(sufficient_sample))
maximal_subset = RestrictionBatch(sufficient_sample)
maximal_subset

42


RestrictionBatch(['AbsI', 'AcvI', 'AgeI', 'ArsI', 'AspJHL3II', 'BoxI', 'Bsp460III', 'Bsp68I', 'BspGI', 'Cfr9I', 'ClaI', 'CspI', 'DraIII', 'Ecl35734I', 'FspAI', 'GauT27I', 'Lmo370I', 'MauBI', 'MluI', 'MreI', 'MspSC27II', 'MteI', 'NotI', 'NpeUS61II', 'PalAI', 'PspXI', 'Pst273I', 'PteI', 'PvuI', 'RgaI', 'RigI', 'RpaB5I', 'SfiI', 'SgrDI', 'SmiI', 'SrfI', 'Sse8387I', 'Ssp714II', 'SstE37I', 'Sth20745III', 'TspARh3I', 'UbaF13I'])

In [94]:
def collect_intervals(record: SeqRecord) -> List[Dict[str, Any]]:
    data_intervals = []
    seq_str = str(record.seq)
    ana = Analysis(maximal_subset, record.seq)
    data_cutpos = []
    for enzyme, positions in ana.with_sites().items():
        for pos in positions:
            data_cutpos.append({"RE": repr(enzyme), "Pos": pos})
    



    # TODO collect only data_cutpos and after that merge it to intervals





    if not data_cutpos:
        Ncount = seq_str.count("N")
        data_intervals.append({
            "Record": record.description,
            "Begin": 0,
            "End": len(record),
            "Lenght": len(record),
            "Ncount": Ncount,
        })
        return data_intervals

    cutpos = pd.DataFrame(data_cutpos).sort_values("Pos")
    cutpos.reset_index(drop=True, inplace=True)

    beg = 0
    prev_RE = None
    for i, row in cutpos.iterrows():
        end = row.Pos
        interval = seq_str[beg: end]
        Ncount = interval.count("N") if len(interval) > 5000 else None
        data_intervals.append({
            "Record": record.description,
            "Begin": beg,
            "End": end,
            "REbegin": prev_RE,
            "REend": row.RE,
            "Lenght": end - beg,
            "Ncount": Ncount,
        })
        beg = end
        prev_RE = row.RE

    # last interval
    end = len(record)
    interval = seq_str[beg: end]
    Ncount = interval.count("N") if len(interval) > 5000 else None
    data_intervals.append({
        "Record": record.description,
        "Begin": beg,
        "End": end,
        "REbegin": prev_RE,
        "REend": None,
        "Lenght": end - beg,
        "Ncount": Ncount,
    })
    return data_intervals


def iterate_over_records(indir: str) -> SeqRecord:
    for fp in glob.glob(os.path.join(indir, "*.fna")):
        for record in SeqIO.parse(fp, format="fasta"):
            yield record

In [ ]:
# 27 min and 10GB of RAM
threads = 24
human_genome = iterate_over_records("../data/external/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/")
with Pool(threads) as p:
    pre_data_intervals = p.map(collect_intervals, human_genome)

data_intervals = []
for xx in pre_data_intervals:
    for x in xx:
        data_intervals.append(x)

intervals = pd.DataFrame(data_intervals)

In [98]:
len(data_intervals)

4038742

In [83]:
pd.DataFrame(data_intervals).sort_values("Lenght").tail(100)

,Record,Begin,End,REbegin,REend,Lenght,Ncount
2061955,"NC_000024.10 Homo sapiens chromosome Y, GRCh38...",56770699,56821668,Ecl35734I,ArsI,50969,50000.0
1653359,"NC_000022.11 Homo sapiens chromosome 22, GRCh3...",18658754,18709750,Ecl35734I,BspGI,50996,50000.0
3617378,"NC_000016.10 Homo sapiens chromosome 16, GRCh3...",36260147,36311205,DraIII,DraIII,51058,50242.0
2333489,"NC_000009.12 Homo sapiens chromosome 9, GRCh38...",60688010,60739122,DraIII,SstE37I,51112,50000.0
908800,"NC_000023.11 Homo sapiens chromosome X, GRCh38...",120878295,120929460,Ecl35734I,SmiI,51165,50000.0
...,...,...,...,...,...,...,...
1638291,"NC_000022.11 Homo sapiens chromosome 22, GRCh3...",0,10510308,None,UbaF13I,10510308,10510000.0
2333359,"NC_000009.12 Homo sapiens chromosome 9, GRCh38...",45514486,60519593,Ecl35734I,BoxI,15005107,15000000.0
0,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",0,16000812,None,AcvI,16000812,16000000.0
2864828,"NC_000001.11 Homo sapiens chromosome 1, GRCh38...",125183279,143188636,ClaI,ClaI,18005357,18000000.0


In [72]:
data_cutpos

[]

In [54]:
record

SeqRecord(seq=Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'), id='NC_000014.9', name='NC_000014.9', description='NC_000014.9 Homo sapiens chromosome 14, GRCh38.p14 Primary Assembly', dbxrefs=[])

In [49]:
ana = Analysis(maximal_subset, record.seq)

In [50]:
data = []
for enzyme, positions in ana.with_sites().items():
    for pos in positions:
        data.append({"RE": repr(enzyme), "Pos": pos})
cutpos = pd.DataFrame(data).sort_values("Pos")
cutpos.reset_index(drop=True, inplace=True)
cutpos.to_csv("../data/interim/Chr1_cutpos.csv", index=None)
cutpos

,RE,Pos
0,AcvI,16000812
1,DraIII,16000815
2,Ecl35734I,16001785
3,ArsI,16003465
4,ArsI,16003497
...,...,...
213518,PteI,106883553
213519,BoxI,106883621
213520,AspJHL3II,106883656
213521,UbaF13I,106883659


In [51]:
cutpos = pd.read_csv("../data/interim/Chr1_cutpos.csv")
cutpos

,RE,Pos
0,AcvI,16000812
1,DraIII,16000815
2,Ecl35734I,16001785
3,ArsI,16003465
4,ArsI,16003497
...,...,...
213518,PteI,106883553
213519,BoxI,106883621
213520,AspJHL3II,106883656
213521,UbaF13I,106883659


In [67]:
data = []
intervals = []
beg = 0
prev_RE = None
for i, row in cutpos.iterrows():
    end = row.Pos
    interval = seq_str[beg: end]
    if len(interval) > 5000:
        Ncount = interval.count("N") if "N" in interval else 0
    else:
        Ncount = None
    data.append({
        "Record": record.description,
        "Begin": beg,
        "End": end,
        "REbegin": prev_RE,
        "REend": row.RE,
        "Lenght": end - beg,
        "Ncount": Ncount,
    })
    beg = end
    prev_RE = row.RE

end = len(record)
interval = seq_str[beg: end]
if len(interval) > 5000:
    Ncount = interval.count("N") if "N" in interval else 0
else:
    Ncount = None
data.append({
    "Record": record.description,
    "Begin": beg,
    "End": end,
    "REbegin": prev_RE,
    "REend": None,
    "Lenght": end - beg,
    "Ncount": Ncount,
})

df = pd.DataFrame(data)

In [69]:
df.sort_values("Lenght", ascending=False).head(30)

,Record,Begin,End,REbegin,REend,Lenght,Ncount
0,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",0,16000812,None,AcvI,16000812,16000000.0
213523,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",106883687,107043718,MreI,None,160031,160000.0
3215,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",18712272,18862751,SmiI,Ecl35734I,150479,150000.0
4976,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",19511251,19611786,Cfr9I,BspGI,100535,100000.0
2358,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",18172920,18224875,Ecl35734I,UbaF13I,51955,50000.0
185,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",16102253,16117148,BspGI,ArsI,14895,7856.0
25707,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",29124968,29133460,SmiI,BspGI,8492,0.0
161,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",16068801,16077232,DraIII,SstE37I,8431,0.0
179,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",16085223,16092685,ArsI,Ecl35734I,7462,2937.0
143299,"NC_000014.9 Homo sapiens chromosome 14, GRCh38...",83193477,83200664,Cfr9I,BspGI,7187,0.0


In [ ]:
# for fp in glob.glob("../data/external/GCF_000001405.40/ncbi_dataset/data/GCF_000001405.40/*.fna"):
#     if "chrMT" in fp:
#         continue
    
#     for record in SeqIO.parse(fp, format="fasta"):
#         for partof_chr in re.split("N{50,}", record):
#             if "mt" in partof_chr.description.lower():
#                 continue
#             ana = Analysis(maximal_subset, partof_chr.seq)
#             for enzyme, positions in ana.with_sites().items():
#                 # use values to sort and write to table
#                 # after that analize table and create subset
#                 pass
            

#         break